In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import os
import glob
import pandas as pd

In [7]:
!pip3 install torch torchvision

  Using cached torch-2.0.1-cp39-none-macosx_10_9_x86_64.whl (143.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.6 MB/s eta 0:00:0000:0100:01


In [2]:
# load image from the IAM database (actually this model is meant to be used on printed text)
url = 'https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/rsoedarnadi/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [4]:
image = Image.open("/Users/rsoedarnadi/Documents/GitHub/bdc-team4/Data Test for BDC 2023 - Penyisihan/DataTest1.png")

In [5]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

/Users/rsoedarnadi/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
labels_csv = pd.read_csv('/Users/rsoedarnadi/Documents/GitHub/bdc-team4/DataTrain.csv', delimiter=';')
list_license_plates = labels_csv['Vehicleregistrationplate']
list_file_names = labels_csv['NameofFile']

In [20]:
generated_plate_numbers = []

In [21]:
for file in list_file_names:
    path = os.path.join(os.getcwd(), 'Train-Data', file)
    image = Image.open(path)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_plate_numbers.append(generated_text)

In [23]:
import re

In [24]:
generated_plate_numbers

['- REG-1',
 '8 1074 * 08',
 '$ 1031.00',
 '8 187, EDA',
 '8 1089* VD',
 'B 1972, RBP',
 'APR2400-U',
 'MR 6268.49',
 'ITEM',
 '8 1554, EJA',
 '8 1160 BH',
 '8 1032, PJE',
 'B 173, PUT',
 '(8.49%.MSN)',
 '8 1254 TFX',
 'B 1618 WKJ',
 'TAX(RM)',
 'BATAT 001',
 'B 1903. RFD',
 '[B 1897, ELS)',
 '8 119 ZB',
 '8 1021 KIF',
 '(8 1566, FAN)',
 '8 1578 SUL',
 '8 9075*08',
 '***',
 '8 1860 EFI',
 '8 1254 TFX',
 '8 1684 BUN',
 'A 1565 E',
 '[8.10%, WSP]',
 '8 1946 TKN',
 'B=1629 SUN]',
 'B 115%',
 'AB 1727 MN',
 '8 1144 ERN',
 'B 1732 PJH',
 'B 1926 MYF',
 '8 1269 EZB',
 'AB 5571 SA',
 '8 1691 THU',
 '8 1289.00',
 '8 1773-U',
 '- AD 8746*OF',
 'A 80% VA',
 'ABKASPTS',
 'B 1295, JJD',
 'B -1078 00',
 'B 1898 STA',
 '@ 114.10',
 'B 1652 UN',
 'B 1819. CKF',
 '(RM)',
 'AF8472987',
 '8 1841 SREF',
 'TAX SUMMARY',
 'A 9192 ZML',
 '0.18 1598,EL0 @',
 'A 8014 VA',
 'B 1312. SIM]',
 'AA 8414.PB',
 '8 11% NO',
 'AD 6259.99',
 'AMPROVING',
 '49 3850.96',
 'DRUGS',
 '8 1739 TZI',
 'B 1541 SEP',
 'BAUGYING

In [26]:
generated_plate_numbers_ = generated_plate_numbers

In [27]:
for i, platenum in enumerate(generated_plate_numbers_):
   generated_plate_numbers_[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

In [29]:
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = "0 %"
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = "100 %"
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = str(round((num_matches / len(actual_plate)), 2) * 100)
            accuracy += "%"
        print("{0:25}{1:25}{2:30}{3:25}".format(name, actual_plate, predict_plate, accuracy))

In [30]:
print("File Name", "\t\t", "Actual License Plate", "\t", "Predicted License Plate", "\t", "Accuracy")
print("--------------------", "\t", "--------------------", "\t", "-----------------------", "\t", "--------")
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers_)

File Name 		 Actual License Plate 	 Predicted License Plate 	 Accuracy
-------------------- 	 -------------------- 	 ----------------------- 	 --------
DataTrain1.png           A7814                    REG1                          20.0%                    
DataTrain2.png           B1074QO                  8107408                       56.99999999999999%       
DataTrain3.png           B1031QO                  103100                        0.0%                     
DataTrain4.png           B187EDA                  8187EDA                       86.0%                    
DataTrain5.png           B1089VD                  81089VD                       86.0%                    
DataTrain6.png           B1972RBP                 B1972RBP                      100 %                    
DataTrain7.png           AB2400WU                 APR2400U                      38.0%                    
DataTrain8.png           AB6268YQ                 MR626849                      50.0%                    
